## Question-2. Part-1 Sequence to Sequence Model for Translation (4pt)

###Loading data files

In [ ]:
# Commented out IPython magic to ensure Python compatibility.
from __future__ import print_function, division
from torch.optim import lr_scheduler
%matplotlib inline
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from google.colab import drive
import heapq

In [ ]:
drive.mount('/content/drive')
%cd /content/drive/MyDrive/IDS576_Assignment_3
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


Mounted at /content/drive
/content/drive/MyDrive/IDS576_Assignment_3


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import re
import random
from torch import optim
import torch.nn.functional as F
import unicodedata
import string
import torch.nn as nn
import torch

In [ ]:
cuda = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
SOS_tkn = 0
EOS_tkn = 1
class Lang:
  def __init__(self, name):
      self.name = name
      self.wordToIndex = {}
      self.wordToCount = {}
      self.indexToword = {0: "SOS", 1: "EOS"}
      self.min_words = 2

  def addSentence(self, sentence):
    for word in sentence.split(' '):
      self.addWord(word)

  def addWord(self, word):
    if word not in self.wordToIndex:
      self.wordToIndex[word] = self.min_words
      self.wordToCount[word] = 1
      self.indexToword[self.min_words] = word
      self.min_words += 1
    else:
      self.wordToCount[word] += 1

def unicodeToAscii(ns):
  return ''.join(c for c in unicodedata.normalize('NFD', ns)
    if unicodedata.category(c) != 'Mn')

def stringNormalize(ns):
  ns = unicodeToAscii(ns.lower().strip())
  ns = re.sub(r"([.!?])", r" \1", ns)
  ns = re.sub(r"[^a-zA-Z.!?]+", r" ", ns)
  return ns

In [ ]:
def readLang(lng_1, lng_2, reverse=False):
    print("Reading Data:")

    lines = open('/content/drive/MyDrive/IDS576_Assignment_3/spa-eng/spa.txt' , encoding='utf-8').\
        read().strip().split('\n')

    pairs = [[stringNormalize(s) for s in l.split('\t')] for l in lines]
    l=[]
    for s in pairs:
      l.append(s[0:2])
    pairs = l
    pairs

    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        inp_lang, op_lang = Lang(lng_2), Lang(lng_1)
    else:
        inp_lang, op_lang = Lang(lng_1), Lang(lng_2)

    return inp_lang, op_lang, pairs

MX_LENGTH = 10

eng_prefixes = (
  "i am ", "i m ",
  "he is", "he s ",
  "she is", "she s ",
  "you are", "you re ",
  "we are", "we re ",
  "they are", "they re "
)
def filterPair(p):
  return len(p[0].split(' ')) < MX_LENGTH and len(p[1].split(' ')) < MX_LENGTH

def filterPairs(pairs):
  return [pair for pair in pairs if filterPair(pair)]

def prepareData(lng_1, lng_2, reverse=False):
    inp_lang, op_lang, pairs = readLang(lng_1, lng_2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        inp_lang.addSentence(pair[0])
        op_lang.addSentence(pair[1])
    print("Counted words:")
    print(inp_lang.name, inp_lang.min_words)
    print(op_lang.name, op_lang.min_words)
    return inp_lang, op_lang, pairs

inp_lang, op_lang, pairs = prepareData('eng', 'spa', True)

Reading Data:
Read 118964 sentence pairs
Trimmed to 90751 sentence pairs
Counting words...
Counted words:
spa 20221
eng 10763


### Encoder and Decoder

In [ ]:
class EncoderRNN(nn.Module):
  def __init__(self, input_size, hdnSize):
    super(EncoderRNN, self).__init__()
    self.hdnSize = hdnSize

    self.embedding = nn.Embedding(input_size, hdnSize)
    self.gru = nn.GRU(hdnSize, hdnSize)

  def forward(self, input, hidden):
    embdInp = self.embedding(input).view(1, 1, -1)
    output = embdInp
    output, hidden = self.gru(output, hidden)
    return output, hidden

  def initHidden(self):
    return torch.zeros(1, 1, self.hdnSize, device=cuda)

class DecoderRNN(nn.Module):
  def __init__(self, hdnSize, output_size):
    super(DecoderRNN, self).__init__()
    self.hdnSize = hdnSize
    self.embedding = nn.Embedding(output_size, hdnSize)
    self.gru = nn.GRU(hdnSize, hdnSize)
    self.out = nn.Linear(hdnSize, output_size)
    self.softmax = nn.LogSoftmax(dim=1)
def forward(self, input, hidden):
    output = self.embedding(input).view(1, 1, -1)
    output = F.relu(output)
    output, hidden = self.gru(output, hidden)
    output = self.softmax(self.out(output[0]))
    return output, hidden
def initHidden(self):
    return torch.zeros(1, 1, self.hdnSize, device=cuda)

In [ ]:
class AttnDecoderRNN(nn.Module):
  def __init__(self, hdnSize, output_size, dropout_p=0.1, max_length=MX_LENGTH):
    super(AttnDecoderRNN, self).__init__()
    self.hdnSize = hdnSize
    self.output_size = output_size
    self.dropout_p = dropout_p
    self.max_length = max_length

    self.embedding = nn.Embedding(self.output_size, self.hdnSize)
    self.attn = nn.Linear(self.hdnSize * 2, self.max_length)
    self.attn_combine = nn.Linear(self.hdnSize * 2, self.hdnSize)
    self.dropout = nn.Dropout(self.dropout_p)
    self.gru = nn.GRU(self.hdnSize, self.hdnSize)
    self.out = nn.Linear(self.hdnSize, self.output_size)

  def forward(self, input, hidden, encoderOutputs):
      embdInp = self.embedding(input).view(1, 1, -1)
      embdInp = self.dropout(embdInp)

      attn_weights = F.softmax(
        self.attn(torch.cat((embdInp[0], hidden[0]), 1)), dim=1)
      attnApplied = torch.bmm(attn_weights.unsqueeze(0), encoderOutputs.unsqueeze(0))

      output = torch.cat((embdInp[0], attnApplied[0]), 1)
      output = self.attn_combine(output).unsqueeze(0)

      output = F.relu(output)
      output, hidden = self.gru(output, hidden)

      output = F.log_softmax(self.out(output[0]), dim=1)
      return output, hidden, attn_weights

  def initHidden(self):
      return torch.zeros(1, 1, self.hdnSize, device=cuda)

In [ ]:
def indexesfSentence(lang, sentence):
  return [lang.wordToIndex[word] for word in sentence.split(' ')]

def tensorfSentence(lang, sentence):
  indexes = indexesfSentence(lang, sentence)
  indexes.append(EOS_tkn)
  return torch.tensor(indexes, dtype=torch.long, device=cuda).view(-1, 1)

def tensorsFromPair(pair):
  inpTensor, target_tensor = tensorfSentence(inp_lang, pair[0]), tensorfSentence(op_lang, pair[1])
  return (inpTensor, target_tensor)

### NN Training

In [ ]:
teacher_forcing_ratio = 0.5

def train(inpTensor, target_tensor, encoder, decoder, encoderOptimizer, decoderOptimizer, criterion, max_length=MX_LENGTH):
  encoderHdn = encoder.initHidden()

  encoderOptimizer.zero_grad()
  decoderOptimizer.zero_grad()
  input_length = inpTensor.size(0)
  target_length = target_tensor.size(0)
  encoderOutputs = torch.zeros(max_length, encoder.hdnSize, device=cuda)
  loss = 0

  for ei in range(input_length):
    encoder_output, encoderHdn = encoder(
      inpTensor[ei], encoderHdn)
    encoderOutputs[ei] = encoder_output[0, 0]

  decoder_input = torch.tensor([[SOS_tkn]], device=cuda)
  decoder_hidden = encoderHdn
  use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

  if use_teacher_forcing:
    for di in range(target_length):
      decoderOpt, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoderOutputs)
      loss += criterion(decoderOpt, target_tensor[di])
      decoder_input = target_tensor[di]
  else:
    for di in range(target_length):
      decoderOpt, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoderOutputs)
      topv, topi = decoderOpt.topk(1)
      decoder_input = topi.squeeze().detach()

      loss += criterion(decoderOpt, target_tensor[di])
      if decoder_input.item() == EOS_tkn:
        break
  loss.backward()

  encoderOptimizer.step()
  decoderOptimizer.step()

  return loss.item() / target_length

In [ ]:
import time
import math
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)
def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0
    plotLossTotal = 0

    encoderOptimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoderOptimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        inpTensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(inpTensor, target_tensor, encoder,
                     decoder, encoderOptimizer, decoderOptimizer, criterion)
        print_loss_total += loss
        plotLossTotal += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plotLossTotal / plot_every
            plot_losses.append(plot_loss_avg)
            plotLossTotal = 0

    showPlot(plot_losses)

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
  plt.figure()
  fig, ax = plt.subplots()
  # this locator puts ticks at regular intervals
  loc = ticker.MultipleLocator(base=0.2)
  ax.yaxis.set_major_locator(loc)
  plt.plot(points)

### Evaluation

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MX_LENGTH):
  with torch.no_grad():
    inpTensor = tensorfSentence(inp_lang, sentence)
    input_length = inpTensor.size()[0]
    encoderHdn = encoder.initHidden()

    encoderOutputs = torch.zeros(max_length, encoder.hdnSize, device=cuda)

    for ei in range(input_length):
      encoder_output, encoderHdn = encoder(inpTensor[ei],
                                                encoderHdn)
      encoderOutputs[ei] += encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_tkn]], device=cuda) # SOS

    decoder_hidden = encoderHdn

    decoded_words = []
    decoder_attentions = torch.zeros(max_length, max_length)

    for di in range(max_length):
      decoderOpt, decoder_hidden, decoder_attention = decoder(
        decoder_input, decoder_hidden, encoderOutputs)
      decoder_attentions[di] = decoder_attention.data
      topv, topi = decoderOpt.data.topk(1)
      if topi.item() == EOS_tkn:
        decoded_words.append('<EOS>')
        break
      else:
        decoded_words.append(op_lang.indexToword[topi.item()])

      decoder_input = topi.squeeze().detach()

    return decoded_words, decoder_attentions[:di + 1]

def randomevaluation(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hdnSize = 256
encoder1 = EncoderRNN(inp_lang.min_words, hdnSize).to(cuda)
attn_decoder1 = AttnDecoderRNN(hdnSize, op_lang.min_words, dropout_p=0.1).to(cuda)

trainIters(encoder1, attn_decoder1, 100000, print_every=10000)

3m 17s (- 29m 40s) (10000 10%) 4.3229
6m 39s (- 26m 36s) (20000 20%) 3.5987
9m 47s (- 22m 51s) (30000 30%) 3.2527
12m 52s (- 19m 18s) (40000 40%) 3.0135
15m 57s (- 15m 57s) (50000 50%) 2.8800
18m 59s (- 12m 39s) (60000 60%) 2.7143
22m 4s (- 9m 27s) (70000 70%) 2.6357
25m 9s (- 6m 17s) (80000 80%) 2.5725
28m 18s (- 3m 8s) (90000 90%) 2.4868
31m 24s (- 0m 0s) (100000 100%) 2.4081


In [ ]:
randomevaluation(encoder1, attn_decoder1)

for i in range(5000, 50000, 5000):
  pair = pairs[i]
  print('>', pair[0])
  print('=', pair[1])
  output_words, attentions = evaluate(encoder1, attn_decoder1, pair[0])
  output_sentence = ' '.join(output_words)
  print('<', output_sentence)
  print('')

def translate(input_sentence):
    output_words, attentions = evaluate(encoder1, attn_decoder1, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))

> preguntale lo que ha hecho .
= ask her what she has done .
< you ve what done done . <EOS>

> no toco el oboe .
= i don t play the oboe .
< i won t play my . . <EOS>

>  cuanto dinero tiene tom ?
= how much money does tom have ?
< how much did tom money ? <EOS>

> es idea de tom .
= it s tom s idea .
< it s idea tom tom . <EOS>

> tom no te vio .
= tom didn t see you .
< tom didn t get you . <EOS>

> mira lo que esta haciendo mary .
= look at what mary is doing .
< i know mary this is mary . . <EOS>

>  puedes ayudarme a levantar esto ?
= can you help me lift this ?
< can you help me this ? ? <EOS>

> tom aseguro que habia visto un ovni .
= tom claimed that he saw a ufo .
< tom thought he had a a . . . <EOS>

> me puedo esconder en cualquier lugar .
= i can hide anywhere .
< i can stay in place in place . <EOS>

> a tom le esta yendo muy bien hoy .
= tom is doing very well today .
< tom will this a very good time . <EOS>

>  que listo !
= that s clever .
< what a ! ! ! ! ! ! ! !

>  

In [ ]:
def translate(input_sentence):
    output_words, attentions = evaluate(encoder1, attn_decoder1, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))

# Translating the sentences

In [ ]:
translate('ella llevaba un bonito sombrero')

input = ella llevaba un bonito sombrero
output = she have a red on time . <EOS>


In [ ]:
translate('disfruta del espectaculo')

input = disfruta del espectaculo
output = let s about your right ? <EOS>


In [ ]:
translate('como estas')

input = como estas
output = you re know how <EOS>


In [ ]:
translate('soy buena')

input = soy buena
output = i m good at good <EOS>


In [ ]:
translate('soy una gran persona')

input = soy una gran persona
output = i m a good person . <EOS>
